In [19]:
import sys
sys.path.append('../')
from data_loader import DataLoader
import random
from feature_selector import BaselineSelector
from feature_selector import FeatureExtractor
import numpy as np

In [25]:
# file path to our dataset to all epoches of all participants
data_path = "../data/dataset"

# create a data loader for all participants
par_loaders = [0] * 88
for i in range(88):
    par_loaders[i] = DataLoader(data_path, [i])

# returns all data of all epochs for participant one 
participants = par_loaders[0][:]
# returns data of the first epoch for participant one for all channels
participant1 = participants[0]['data']

(19, 2500)

In [37]:
selector1 = BaselineSelector()
selector1.selectFeatures(['mean'])

channel_means = [[]] * 19
channel_means[0].append(1)
channel_means

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [ ]:
for participant in par_loaders:
    for epoch in participant[:]:
        for channel in range(19):
            epoch_data = epoch['data'][channel]
            out1 = selector1.transform(epoch_data)
            extractor = FeatureExtractor()
            mean_par = np.mean(out1)

    means.append(mean_par)
print(means)